#Build model using data from file

We use the `pandas` module to read in a large datafile that we then use to create a problem. The datafile contains the edges of a directed graph $G = (V,A)$ and we will use the data to create a **shortest-path problem** from the first to the last node (sorted by their index). The format of the data is: `i,j,c` where i and j are the end nodes of the directed $i \to j$ arc, while $c$ is the cost of the arc.

In [ ]:
pip install xpress

     |████████████████████████████████| 43.8MB 88kB/s 


In [ ]:
import xpress as xp
import pandas as pd

Using the Community license in this session. If you have a full Xpress license, first set the XPAUTH_PATH environment variable to the full path to your license file, xpauth.xpr, and then restart Python. If you want to use the FICO Community license and no longer want to see this message, set the XPAUTH_PATH environment variable to: /usr/local/lib/python3.6/dist-packages/xpress/license/community-xpauth.xpr
NB: setting XPAUTH_PATH will also affect any other Xpress products installed on your system.


In [ ]:
#read graph data
df = pd.read_csv('jupyter_2020-09-16_wednesday_graph_data.csv')

In [ ]:
tail = df['i']
head = df['j']
cost = df['c']

In [ ]:
#V is created as a set
V = set(tail) | set(head) #get set V of nodes

#create A as a dictionary with cost as value
A = {(tail[i],head[i]) : cost[i] for i, _ in enumerate(tail)}

source = min(V)
dest = max(V)

#Let's print out V,A, source and dest

print("Set of nodes:", V)
print("Arcs:", A.keys())
print("Find the shortest path from {0} to {1}".format(source, dest))

Set of nodes: {1, 2, 3, 4, 5}
Arcs: dict_keys([(1, 2), (2, 1), (1, 3), (3, 1), (1, 4), (4, 1), (1, 5), (5, 1), (2, 3), (3, 2), (2, 4), (4, 2), (2, 5), (3, 4), (4, 3), (3, 5), (5, 3), (4, 5), (5, 4)])
Find the shortest path from 1 to 5


In [ ]:
#Use the vars() functions to specify an index set for the variables
x = xp.vars(list(A.keys()), vartype = xp.binary)

#The right-hand side is also a dictionary. It specifies the
#flow balance, i.e. the difference between outgoing and incoming
# flow at a node. It must be 1 at the source, -1 at destination, and 
# 0 at each intermediate node.

b = {i: 0 for i in V}
b[source] = 1
b[dest] = -1

#conservation of flow: The total outgoing flow minus the total incomming flow 
#must equal the above RHS, for each node in V
conservation = [xp.Sum(x[(i,j)] for j in V if (i,j) in A.keys()) - \
                xp.Sum(x[(j,i)] for j in V if (j,i) in A.keys()) == b[i] for i in V]

#The objective function is the length of the path, i.e. the weighted sum of all arc variables
obj = xp.Sum(A[(i,j)] * x[(i,j)] for (i,j) in A)

p = xp.problem(x,conservation, obj)

p.solve()

for (i,j) in A:
  if p.getSolution(x[(i,j)]) > 0.5:
    print(i,j)

FICO Xpress v8.11.0, Community, solve started 9:01:42, Nov 20, 2020
Heap usage: 341KB (peak 341KB, 559KB system)
Minimizing MILP noname with these control settings:
OUTPUTLOG = 1

Original problem has:
         5 rows           19 cols           38 elements        19 globals
Presolved problem has:
         5 rows           19 cols           38 elements        19 globals
Presolve finished in 0 seconds
Heap usage: 370KB (peak 371KB, 561KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 2.00e+00,  1.90e+01] / [ 2.00e+00,  1.90e+01]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 11.9GB
 *** Heuristic solution found:    19.000000      Time: 0 ***
 *** Heuristic solution found:     9.000000      Time: 0 ***
Starting concurre

#Indicator Constraints:

Indicator constraints are constraints that must hold if a given binary variable is equal to one, and they are not enforced otherwise. For instance, if a problem has two variables x and y and we want to constrain them to $x + 2y \geq 4$ only if a third binary variable z is set to one, then this can be denoted as

$z = 1 \rightarrow x + 2y \geq 4$

Indicator contraints are common in Mixed Integer Linear Optimization. In the Xpress python interface, the above indicator constrain would be added to a problem as follows:
p.addIndicator ( z == 1, x + 2*y >= 4)

We can specify indicators with the `z ==0` condition too, but there are only two alternatives; constraints other that `z==0` or `z==1` as the first argument of an indicator will trigger an error.

p.addIndicator (z == 0, x + 2*y > = 4)

In [ ]:
x = xp.var()
y = xp.var()

p = xp.problem(x,y, xp.abs(x-y) + 2*xp.max(2*x + 4, 3*y + 2*x),
               x+2*y >= 20,
               x + 3*y <= 25)

p.solve()

print(p.getSolution(x), p.getSolution(y))


Original problem size
   linear:    3 rows, 4 columns, 5 linear coefficients
   nonlinear: 1 coefficients, 23 tokens
Nonlinear presolve
   compressing formula space (in use\total : 24\49)
   removed 2 formulas and 24 formula tokens
   checking equivalence to general constraints ABS/MIN/MAX and PWL reformulations
   converted 1 ABS to optimizer MIP constructs
   converted 1 MIN/MAX to optimizer MIP constructs
   converted 4 formulas to linear constraints
   converted objective transfer row to linear objective
   problem is equivalent, reformulated as a MIP problem
Presolved problem size
   linear:    6 rows, 9 columns, 12 linear coefficients
Problem is nonlinear presolved
Minimizing problem using Xpress-Optimizer
FICO Xpress v8.11.0, Community, solve started 9:46:15, Nov 20, 2020
Heap usage: 688KB (peak 754KB, 942KB system)
Minimizing MILP noname with these control settings:
OUTPUTLOG = 1
IFCHECKCONVEXITY = 0

Original problem has:
         6 rows            9 cols           12 element

#Manual MIP reformulation of $|x|$, max/min, and ***AND/OR***



In the above problem, we defined a quantity $|x-y|$ with the `xpress.abs` operator.

Below is how we would have to model the constraint $z = |x-y|$ if we did not have the `xp.abs` function. We would have to model $z = |x-y|$ by introducing a binary variable $w$ and writing the following constraints, two of which are indicator constraints:

$$
\begin{array}{lll}
z \ge x-y\\
z \ge y-x\\
w = 1 \Rightarrow z \le x-y\\
w = 0 \Rightarrow z \le y-x\\
w \in \{0,1\}
\end{array}
$$

If, however, we only had the *inequality* constraint $z \ge |x-y|$, then reformulation would be a lot easier: the two constraints

$$
z \ge x - y; \qquad z \ge y-x
$$

would suffice.

And/Or operator do not require extra variable: for an operation $z = x \wedge y$, with $x$ and $y$ binary variables, one can write

$$
\begin{array}{l}
z \le x\\
z \le y\\
z \ge x + y - 1
\end{array}
$$

and similar for an operation $z = x \vee y$:

$$
\begin{array}{l}
z \ge x\\
z \ge y\\
z \le x + y
\end{array}
$$

Note that $z\le x$ above works like an implication: it is equivalent to $z = 1 \Rightarrow x = 1$, and therefore also an implication in the opposite direction (from $x$ to $z$): $x = 0 \Rightarrow z = 0$.

In [ ]:
x = xp.var(vartype = xp.binary)
y = xp.var(vartype = xp.binary)
z = xp.vars(10, vartype = xp.binary)

p = xp.problem(x,y,z,
               (x&y) + x | y | xp.Or(z[0], z[1], z[4]) == 1,
               xp.Sum(z))

p.solve()


Original problem size
   linear:    1 rows, 13 columns, 0 linear coefficients
   nonlinear: 1 coefficients, 16 tokens
Nonlinear presolve
   checking equivalence to general constraints ABS/MIN/MAX and PWL reformulations
   converted 4 MIN/MAX to optimizer MIP constructs
   converted 2 formulas to linear constraints
   problem is equivalent, reformulated as a MIP problem
Presolved problem size
   linear:    2 rows, 18 columns, 4 linear coefficients
Problem is nonlinear presolved
Minimizing problem using Xpress-Optimizer
FICO Xpress v8.11.0, Community, solve started 12:48:06, Nov 20, 2020
Heap usage: 686KB (peak 752KB, 947KB system)
Minimizing MILP noname with these control settings:
OUTPUTLOG = 1
IFCHECKCONVEXITY = 0

Original problem has:
         2 rows           18 cols            4 elements        12 globals
         4 gencons
Presolved problem has:
         5 rows            5 cols           13 elements         4 globals
LP relaxation tightened
Presolve finished in 0 seconds
Heap u

**Exercise**: Formulate a MIP to solve a SATisfaction problem defined by the clauses on three variables as defined below, where $x_i$ means a variable is included as-is and $\bar x_i$ means its negation:

$$
\begin{array}{lllll}
  \bar x_1 &\vee&      x_2 &    &          \\
           &    & \bar x_2 &\vee&      x_3 \\
  \bar x_1 &    &          &\vee&      x_3 \\
           &    & \bar x_2 &\vee& \bar x_3 \\
       x_1 &    &          &\vee&      x_3 \\
       x_1 &\vee& \bar x_2 &    &          \\
\end{array}
$$

In [ ]:
# Identify each clause with -1 if variable is negated, 1 if non-negated, 0 if not present.

id_clauses = [
              [-1,1,0],
              [0,-1,1],
              [-1,0,1],
              [0,-1,-1],
              [1,0,1],
              [1,-1,0]]
I = [0,1,2]

x = [xp.var(vartype = xp.binary) for _ in I]

clauses = [
           [(x[j] if id[j] == 1 else (1-x[j]) if id[j] == -1 else 0)
           for j in I]
           for id in id_clauses
]

SATcon = [xp.Or(*clause) == 1 for clause in clauses]
p = xp.problem(x,SATcon) #note: no objective function is necessary

p.solve()
print(p.getSolution())


Original problem size
   linear:    6 rows, 4 columns, 0 linear coefficients
   nonlinear: 6 coefficients, 48 tokens
Nonlinear presolve
   checking equivalence to general constraints ABS/MIN/MAX and PWL reformulations
   converted 6 MIN/MAX to optimizer MIP constructs
   converted 12 formulas to linear constraints
   problem is equivalent, reformulated as a MIP problem
Presolved problem size
   linear:    12 rows, 16 columns, 18 linear coefficients
Problem is nonlinear presolved
Minimizing problem using Xpress-Optimizer
FICO Xpress v8.11.0, Community, solve started 14:11:06, Nov 20, 2020
Heap usage: 687KB (peak 753KB, 951KB system)
Minimizing MILP noname with these control settings:
OUTPUTLOG = 1
IFCHECKCONVEXITY = 0

Original problem has:
        12 rows           16 cols           18 elements         3 globals
         6 gencons
Presolved problem has:
         0 rows            0 cols            0 elements         0 globals
LP relaxation tightened
Presolve finished in 0 seconds
Heap

#Simple cutting plane algorithm
We'll implement a very simple cutting plane algorithm. Consider the problem

$$
\begin{array}{ll}
\min &\sum_{i=1}^n\sum_{j=1}^n q_{ij} x_i x_j + \sum_i c_i x_i\\
\textrm{s.t.} & x_i\in \{0,1\} & \forall i=1,2,\ldots, n.
\end{array}
$$

This problem is a **Binary Box-constrained Quadratic Programming problem**: it has a quadratic objective function and all variables are negative; furthermore, the only constraints are the implicit bounds on binary variables, i.e. $\{0,1\}$. The problem can be written in matricial form as follows:

$$
\begin{array}{ll}
\min & x^T Q x + c^T x\\
\textrm{s.t.} & x \in \{0,1\}^n
\end{array}
$$

where $Q$ is a symmetric matrix. In general, the above problem is much harder if the term $x^T Q x$ is nonconvex, i.e., if $Q$ is **not** positive semidefinite. Let's try an example.

In [ ]:
# 2BCDC
import numpy as np
np.random.seed(1234567)

n = 30
Q = np.random.random((n,n)) - .5
x = xp.vars(n, vartype = xp.binary) 
c = np.random.random(n) - .5

p = xp.problem(x,xp.Dot(x,Q,x) + xp.Dot(c,x))

p.solve()


FICO Xpress v8.11.0, Community, solve started 14:34:23, Nov 20, 2020
Heap usage: 356KB (peak 356KB, 636KB system)
Minimizing MIQP noname with these control settings:
OUTPUTLOG = 1

Original problem has:
         0 rows           30 cols            0 elements        30 globals
       900 qobjelem
Presolved problem has:
       657 rows          465 cols         1527 elements        30 globals
Presolve finished in 0 seconds
Heap usage: 655KB (peak 899KB, 638KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [      0.0,       0.0] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 4.33e-03,  4.81e-01] / [ 2.13e-04,  9.83e-01]
  Quadratic      [min,max] : [ 2.13e-04,  9.83e-01] / [      0.0,       0.0]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 11.7GB
Starting concurrent solve with dual

 Co

The Xpress Optimizer can solve such problems after a *reformulation* that does the following:

* Creates a (binary) variable $y_{ij}$ for each product $x_i x_j$;
* For each variable $y_{ij}$, it adds the following constraints linking $y$ with $x$ variables:

$$
\begin{array}{lll}
y_{ij} &\le& x_i \\
y_{ij} &\le& x_j \\
y_{ij} &\ge& x_i + x_j - 1
\end{array}
$$

After this reformulation, the problem becomes as follows:

$$
\begin{array}{ll}
\min & \sum_{i=1}^n \sum_{j=1}^n q_{ij} y_{ij} + \sum_{i=1}^n c_i x_i\\
\textrm{s.t.} &y_{ij} \le x_i  &\forall i,j\\
&y_{ij} \le x_j  &\forall i,j\\
&y_{ij} \ge x_i + x_j - 1 &\forall i,j\\
&x_i \in \{0,1\} \forall i=1,2,\ldots,n\\
&y_{ij} \in \{0,1\} \forall i,j=1,2,\ldots,n\\
\end{array}
$$

We can do this reformulation explicitly by introducing the variables $y_{ij}$. Below we use NumPy vectors of variables as they come in handy to write constraints.

In [ ]:
# 2BCDC

import xpress as xp
import numpy as np

np.random.seed(1234567)

n = 30
Q = np.random.random((n,n)) - 0.5
c = np.random.random(n) - 0.5

x = xp.vars(n, vartype=xp.binary)
Y = xp.vars(n, n, vartype=xp.binary)

p = xp.problem(x, Y, xp.Sum(Q*Y) + xp.Dot(c,x),
               Y == Y.transpose(),
               Y <= x,
               Y.transpose() <= x,
               [Y[i,j] >= x[i] + x[j] - 1 for i in range(n) for j in range(n)])
p.solve()




FICO Xpress v8.11.0, Community, solve started 14:38:58, Nov 20, 2020
Heap usage: 1689KB (peak 1689KB, 1575KB system)
Minimizing MILP noname with these control settings:
OUTPUTLOG = 1

Original problem has:
      3600 rows          930 cols         8010 elements       930 globals
Presolved problem has:
      1305 rows          465 cols         3045 elements       465 globals
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 2034KB (peak 3519KB, 1577KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  2.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 7.60e-04,  5.00e-01] / [ 2.13e-04,  9.83e-01]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 11.7GB
 *** Heuristic solution found:      .000000      Time: 0 ***
 *** Heuristic solution found:     -.107342   